In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
#Read CSV
df = pd.read_csv('data/cleanData.csv')
df.head()

,Unnamed: 0,1-ZCRm,4-SpectralCentroidm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,71-BPMessentia,class
0,0,0.136440,0.262825,1.114423,0.007003,0.256682,128,BigRoom
1,1,0.117039,0.247657,1.065668,0.005387,0.199821,126,BigRoom
2,2,0.085308,0.217205,0.789647,0.008247,0.156822,129,BigRoom
3,3,0.103050,0.233593,0.967082,0.006571,0.168083,129,BigRoom
4,4,0.151730,0.293730,1.353005,0.003872,0.292055,129,BigRoom


In [3]:
# Remove Space for `FALSE POSITIVE` category
# mask = df["koi_disposition"] == "FALSE POSITIVE"
# df.loc[mask, "koi_disposition"] = "False_Positive"
# df["koi_disposition"]

# Create a Train Test Split

Use `class` for the y values

In [4]:
from sklearn.model_selection import train_test_split
y = df["class"]
X = df.drop(columns=["class"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [5]:
X_train.head()

,Unnamed: 0,1-ZCRm,4-SpectralCentroidm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,71-BPMessentia
1758,1758,0.091103,0.229035,0.690514,0.012045,0.173285,123
1401,1401,0.072794,0.239562,0.601325,0.008465,0.107336,126
1545,1545,0.158472,0.294533,1.345575,0.006274,0.287489,121
155,155,0.123627,0.235015,1.020074,0.011133,0.207733,127
744,744,0.079600,0.211973,0.612767,0.017219,0.131065,124


# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [6]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [8]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


In [9]:
encoded_y_train

array([17, 14, 15, ...,  3, 19,  3])

In [10]:
y_train_categorical.shape

(1725, 23)

# Create and train the Deep Learning Model using Keras

In [11]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=7))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=23, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 23)                2323      
Total params: 13,223
Trainable params: 13,223
Non-trainable params: 0
_________________________________________________________________


In [14]:
X_train_scaled.shape


(1725, 7)

In [15]:
y_train_categorical.shape

(1725, 23)

In [16]:
encoded_y_train.shape

(1725,)

In [17]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
 - 1s - loss: 3.0518 - acc: 0.0951
Epoch 2/60
 - 0s - loss: 2.7523 - acc: 0.1896
Epoch 3/60
 - 0s - loss: 2.3517 - acc: 0.2667
Epoch 4/60
 - 0s - loss: 2.0545 - acc: 0.3710
Epoch 5/60
 - 0s - loss: 1.8639 - acc: 0.4093
Epoch 6/60
 - 0s - loss: 1.7198 - acc: 0.4719
Epoch 7/60
 - 0s - loss: 1.6080 - acc: 0.4887
Epoch 8/60
 - 0s - loss: 1.5161 - acc: 0.5223
Epoch 9/60
 - 0s - loss: 1.4327 - acc: 0.5577
Epoch 10/60
 - 0s - loss: 1.3570 - acc: 0.5762
Epoch 11/60
 - 0s - loss: 1.3058 - acc: 0.5728
Epoch 12/60
 - 0s - loss: 1.2458 - acc: 0.6041
Epoch 13/60
 - 0s - loss: 1.2005 - acc: 0.6232
Epoch 14/60
 - 0s - loss: 1.1575 - acc: 0.6313
Epoch 15/60
 - 0s - loss: 1.1171 - acc: 0.6562
Epoch 16/60
 - 0s - loss: 1.0857 - acc: 0.6475
Epoch 17/60
 - 0s - loss: 1.0571 - acc: 0.6539
Epoch 18/60
 - 0s - loss: 1.0211 - acc: 0.6893
Epoch 19/60
 - 0s - loss: 1.0053 - acc: 0.6528
Epoch 20/60
 - 0s - loss: 0.9704 - acc: 0.6910
Epoch 21/60
 - 0s - l

In [18]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.3731 - acc: 0.9130
Normal Neural Network - Loss: 0.3731373781743257, Accuracy: 0.9130434989929199


In [19]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [20]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['ElectronicaDowntempo' 'BigRoom' 'Dance' 'Breaks' 'HipHop']
Actual Labels: ['ElectronicaDowntempo', 'Breaks', 'Dance', 'Breaks', 'HipHop']
